In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [2]:
scaler = StandardScaler()
data = pd.read_csv('/kaggle/input/english-premier-league/final_dataset.csv')
print(data.columns)
data = data.dropna()
# Example: Convert 'Date' column to a datetime object
numerical_columns = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts']
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])
data = data.drop(['HomeTeam', 'AwayTeam','HM1', 'HM2', 'HM3','HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5','HTFormPtsStr','ATFormPtsStr','FTR','Date'], axis=1)
categorical_columns = ['HomeTeam', 'AwayTeam', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5', 'HTFormPtsStr', 'ATFormPtsStr']
# Apply one-hot encoding
#df = pd.get_dummies(df, columns=categorical_columns)
print(data.head(5))
data['Result'] = np.where(data['FTHG'] > data['FTAG'], 1, np.where(data['FTHG'] == data['FTAG'], 0, -1))
X = data.drop(['Result'], axis=1)
y = data['Result']
X_scaled = scaler.fit_transform(X)
n_components = 10  # Choose the number of components
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained variance ratio:", explained_variance_ratio)

Index(['Unnamed: 0', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
       'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5', 'MW', 'HTFormPtsStr',
       'ATFormPtsStr', 'HTFormPts', 'ATFormPts', 'HTWinStreak3',
       'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5', 'ATWinStreak3',
       'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5', 'HTGD', 'ATGD',
       'DiffPts', 'DiffFormPts'],
      dtype='object')
   Unnamed: 0      FTHG      FTAG      HTGS      ATGS      HTGC      ATGC  \
0           0  1.905132 -1.005139 -1.421452 -1.430605 -1.493735 -1.490023   
1           1  1.905132  0.773454 -1.421452 -1.430605 -1.493735 -1.490023   
2           2 -0.406440  1.662751 -1.421452 -1.430605 -1.493735 -1.490023   
3           3  0.364084  0.773454 -1.421452 -1.430605 -1.493735 -1.490023   
4           4  0.364084 -1.005139 -1.421452 -1.430605 -1.493735 -1.490023   

        HTP       ATP   MW  ...  HTLossStr

In [3]:
data.head()

,Unnamed: 0,FTHG,FTAG,HTGS,ATGS,HTGC,ATGC,HTP,ATP,MW,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,Result
0,0,1.905132,-1.005139,-1.421452,-1.430605,-1.493735,-1.490023,-2.280523,-2.345017,1.0,...,0,0,0,0,0,0.013972,-0.020983,0.02656,0.044279,1
1,1,1.905132,0.773454,-1.421452,-1.430605,-1.493735,-1.490023,-2.280523,-2.345017,1.0,...,0,0,0,0,0,0.013972,-0.020983,0.02656,0.044279,1
2,2,-0.406440,1.662751,-1.421452,-1.430605,-1.493735,-1.490023,-2.280523,-2.345017,1.0,...,0,0,0,0,0,0.013972,-0.020983,0.02656,0.044279,-1
3,3,0.364084,0.773454,-1.421452,-1.430605,-1.493735,-1.490023,-2.280523,-2.345017,1.0,...,0,0,0,0,0,0.013972,-0.020983,0.02656,0.044279,-1
4,4,0.364084,-1.005139,-1.421452,-1.430605,-1.493735,-1.490023,-2.280523,-2.345017,1.0,...,0,0,0,0,0,0.013972,-0.020983,0.02656,0.044279,1


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [5]:
# Single-layer perceptron (Simple neural network with one layer)
perceptron = Perceptron(max_iter=1000, random_state=42)
perceptron.fit(X_train, y_train)
perceptron_predictions = perceptron.predict(X_test)

In [6]:
perceptron_accuracy = accuracy_score(y_test, perceptron_predictions)
print(perceptron_accuracy)

0.9583333333333334


In [7]:
## Delta Rule
learning_rate = 0.1
epochs = 100
threshold = 0.5 # Define a threshold for classifying the predictions as 1, 0, or -1

# Initialize weights with small random values
weights = np.random.rand(X_train.shape[1])

# Implement the delta rule algorithm
for epoch in range(epochs):
    for i in range(X_train.shape[0]):
        # (y_pred) using the current weights
        y_pred = np.dot(X_train[i], weights)

        # Compute the error
        error = y_train.iloc[i] - y_pred

        # weight updation
        weights = weights + learning_rate * error * X_train[i]

# Use the trained weights to make predictions on the test data
predictions = np.dot(X_test, weights)
predicted_labels = np.where(predictions > threshold, 1, np.where(predictions < -threshold, -1, 0))

# Evaluate the accuracy of the predictions
delta_rule_accuracy = np.mean(predicted_labels == y_test.values)
print("Delta rule Accuracy:", delta_rule_accuracy)

Delta rule Accuracy: 0.0


In [8]:
## Competitive learing
num_clusters = 3
cluster_centers = X_train[np.random.choice(X_train.shape[0], num_clusters, replace=False)]
learning_rate = 0.1
epochs = 100

# Implement competitive learning algorithm
for epoch in range(epochs):
    for i in range(X_train.shape[0]):
        # Compute distances from the data point to cluster centers
        distances = np.linalg.norm(X_train[i] - cluster_centers, axis=1)

        # Find the index of the winning cluster (the one with the minimum distance)
        winning_cluster_index = np.argmin(distances)

        # Update the winning cluster's center
        cluster_centers[winning_cluster_index] += learning_rate * (X_train[i] - cluster_centers[winning_cluster_index])

# Assign each data point to the closest cluster
cluster_assignments = np.argmin(np.linalg.norm(X_test[:, np.newaxis] - cluster_centers, axis=2), axis=1)

# Evaluate the accuracy of the cluster assignments
competitive_accuracy = np.mean(cluster_assignments == y_test.values)
print("Competitive learing Accuracy:", competitive_accuracy)

Competitive learing Accuracy: 0.04239766081871345


In [9]:
# MLP (Multi-layer perceptron)
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
mlp_predictions = mlp.predict(X_test)

In [10]:
mlp_accuracy = accuracy_score(y_test, mlp_predictions)
print(mlp_accuracy)

0.9992690058479532


In [11]:
if (perceptron_accuracy<mlp_accuracy):
  print("Multi-layer perceptron is better")
elif (mlp_accuracy<perceptron_accuracy):
  print("Simple neural network with one layer is better")
else:
  print("Both are performing equally")

Multi-layer perceptron is better


In [12]:
print('MLP with different Activation function')
# MLP with sigmoid
mlp_sigmoid = MLPClassifier(hidden_layer_sizes=(64, 32), activation='logistic', max_iter=1000, random_state=42)
mlp_sigmoid.fit(X_train, y_train)
mlp_sigmoid_predictions = mlp_sigmoid.predict(X_test)
mlp_sigmoid_accuracy = accuracy_score(y_test, mlp_sigmoid_predictions)
# MLP with Tanh
mlp_tanh = MLPClassifier(hidden_layer_sizes=(64, 32), activation='tanh', max_iter=1000, random_state=42)
mlp_tanh.fit(X_train, y_train)
mlp_tanh_predictions = mlp_tanh.predict(X_test)
mlp_tanh_accuracy = accuracy_score(y_test, mlp_tanh_predictions)
# Create an MLP classifier with ReLU activation
mlp_relu = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', max_iter=1000, random_state=42)
mlp_relu.fit(X_train, y_train)
mlp_relu_predictions = mlp_relu.predict(X_test)
mlp_relu_accuracy = accuracy_score(y_test, mlp_relu_predictions)

# Create an MLP classifier with Leaky ReLU activation
mlp_leaky_relu = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', alpha=0.01, max_iter=1000, random_state=42)
mlp_leaky_relu.fit(X_train, y_train)
mlp_leaky_relu_predictions = mlp_leaky_relu.predict(X_test)
mlp_leaky_relu_accuracy = accuracy_score(y_test, mlp_leaky_relu_predictions)

# Compare the accuracies
print("\nAccuracy Comparison:")
print("MLP with Sigmoid Accuracy: ", mlp_sigmoid_accuracy)
print("MLP with Tanh Accuracy: ", mlp_tanh_accuracy)
print("MLP with ReLU Accuracy: ", mlp_relu_accuracy)
print("MLP with Leaky ReLU Accuracy: ", mlp_leaky_relu_accuracy)

MLP with different Activation function

Accuracy Comparison:
MLP with Sigmoid Accuracy:  0.9992690058479532
MLP with Tanh Accuracy:  0.9985380116959064
MLP with ReLU Accuracy:  0.9992690058479532
MLP with Leaky ReLU Accuracy:  0.9992690058479532


In [15]:
from keras.optimizers import SGD, Adam
# MLP with sgd optimizer
mlp_sgd = MLPClassifier(hidden_layer_sizes=(64, 32), activation='logistic', max_iter=1000, random_state=42, solver='sgd', learning_rate='constant', learning_rate_init=0.001, momentum=0.9)
mlp_sgd.fit(X_train, y_train)
mlp_sgd_predictions = mlp_sgd.predict(X_test)
mlp_sgd_accuracy = accuracy_score(y_test, mlp_sgd_predictions)

# MLP with Adam optimizer
mlp_adam = MLPClassifier(hidden_layer_sizes=(64, 32), activation='logistic', max_iter=1000, random_state=42, solver='adam', learning_rate='constant', learning_rate_init=0.001)
mlp_adam.fit(X_train, y_train)
mlp_adam_predictions = mlp_adam.predict(X_test)
mlp_adam_accuracy = accuracy_score(y_test, mlp_adam_predictions)

# Print accuracy for both optimizers
print("Accuracy with RMSprop optimizer:", mlp_sgd_accuracy)
print("Accuracy with Adam optimizer:", mlp_adam_accuracy)

Accuracy with RMSprop optimizer: 0.9700292397660819
Accuracy with Adam optimizer: 0.9992690058479532
